# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [1]:
import cudf
import cuml

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [2]:
gdf = cudf.read_csv('./data/week3.csv')

In [4]:
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [3]:
gdf['infected'] = gdf['infected'].astype('float32')

Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [5]:
group = gdf.groupby('employment')
rate = group['infected'].sum() / group['infected'].count()
rate.sort_values()

employment
U          0.000217
F          0.003182
H          0.003388
B, D, E    0.003774
A          0.003853
C          0.003882
J          0.003939
X          0.004539
K          0.004772
M          0.004777
N          0.004784
G          0.004948
L          0.004970
O          0.005284
R, S, T    0.005390
Z          0.005655
P          0.006190
V          0.007590
I          0.010354
Q          0.012756
Name: infected, dtype: float64

Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [6]:
code_guide = cudf.read_csv('./data/code_guide.csv')
code_guide

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [8]:
gdf_sex = gdf.groupby(['employment','sex'])
rate = gdf_sex['infected'].sum() / gdf_sex['infected'].count()
rate.sort_values()

employment  sex
U           m      0.000110
            f      0.000329
L           m      0.002197
X           m      0.002435
B, D, E     m      0.002462
H           m      0.002482
K           m      0.002490
A           m      0.002514
M           m      0.002520
F           m      0.002535
N           m      0.002538
R, S, T     m      0.002542
Z           m      0.002543
O           m      0.002543
J           m      0.002546
C           m      0.002569
P           m      0.002577
G           m      0.002596
V           m      0.003685
I           m      0.005117
Q           m      0.005120
L           f      0.007221
H           f      0.007385
N           f      0.007389
X           f      0.007391
A           f      0.007491
G           f      0.007556
F           f      0.007577
P           f      0.007584
Z           f      0.007629
C           f      0.007630
J           f      0.007645
M           f      0.007645
K           f      0.007672
O           f      0.007719
R, S

In [26]:
code_guide.head()

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"


In [31]:
code_guide.rename(columns={'Code': 'employment'}, inplace = True)
code_guide_rate = code_guide.merge(cudf.DataFrame(rate).reset_index(), on=['employment'], how='left')
code_guide_rate.sort_values(by = 'infected')

,employment,Field,sex,infected
1,U,Student,m,0.000109784
0,U,Student,f,0.000329444
27,L,Real estate activities,m,0.002196769
4,X,Outside the UK or not specified,m,0.002434716
12,"B, D, E","Mining, energy and water supply",m,0.0024616
18,H,Transport & storage,m,0.002482171
25,K,Financial & insurance activities,m,0.002489851
9,A,"Agriculture, forestry & fishing",m,0.002514249
29,M,"Professional, scientific & technical activities",m,0.002519579
39,F,Construction,m,0.002535045


## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)